<a href="https://colab.research.google.com/github/sunshineluyao/UTXO/blob/main/UTXO_Age_Distribution_Data_queried_from_public_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.cloud import bigquery

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'crypto-291811'
client = bigquery.Client(project=project_id)

for dataset in client.list_datasets():
  print(dataset.dataset_id)

UTXO


In [ ]:
# TODO(developer): Set table_id to the ID of the destination table.
### Refers to 
#1. result large results:
#https://cloud.google.com/bigquery/docs/writing-results
#2. Export to dataframe
#https://cloud.google.com/bigquery/docs/exporting-data
#ARRAY_TO_STRING(addresses,'.') AS addresses	
table_id_inputs = "crypto-291811.UTXO.inputs"

job_config = bigquery.QueryJobConfig(destination=table_id_inputs)

sql = """
  SELECT 
    spent_transaction_hash,
    spent_output_index,	
    block_timestamp AS spent_block_timestamp,   

  FROM `bigquery-public-data.crypto_bitcoin.inputs`
"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.


print("Query results loaded to the table {}".format(table_id_inputs))

In [ ]:
# TODO(developer): Set table_id to the ID of the destination table.
### Refers to 
#1. result large results:
#https://cloud.google.com/bigquery/docs/writing-results
#2. Export to dataframe
#https://cloud.google.com/bigquery/docs/exporting-data
table_id_outputs = "crypto-291811.UTXO.outputs"

job_config = bigquery.QueryJobConfig(destination=table_id_outputs)

sql = """
  SELECT 
    transaction_hash,
    block_timestamp,
  	index,	
    value
  FROM `bigquery-public-data.crypto_bitcoin.outputs`
"""

# Start the query, passing in the extra configuration.
query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.


print("Query results loaded to the table {}".format(table_id_outputs))

In [18]:
# TODO(developer): Set table_id to the ID of the destination table.
### Refers to 
#1. result large results:
#https://cloud.google.com/bigquery/docs/writing-results
#2. Export to dataframe
#https://cloud.google.com/bigquery/docs/exporting-data

table_id_joint = "crypto-291811.UTXO.joint"

job_config = bigquery.QueryJobConfig(destination=table_id_joint)

sql = """
  SELECT
    outputs.value,  
    outputs.transaction_hash,
    inputs.spent_transaction_hash,
    outputs.index,
    inputs.spent_output_index,
    outputs.block_timestamp,
    inputs.spent_block_timestamp,
   
  FROM `crypto-291811.UTXO.outputs` AS outputs
  LEFT JOIN `crypto-291811.UTXO.inputs` AS inputs
  ON outputs.transaction_hash=inputs.spent_transaction_hash  AND outputs.index = inputs.spent_output_index
  LIMIT 100

"""

# Start the query, passing in the extra configuration.
query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.


print("Query results loaded to the table {}".format(table_id_joint))

Query results loaded to the table crypto-291811.UTXO.joint


In [ ]:
### For interaction with google sheets, refers to 
#1.Connecting with google Sheets: gsread https://colab.research.google.com/notebooks/snippets/sheets.ipynb#scrollTo=6d0xJz3VzLOo
#2.Interacting with google sheets: df2gspread https://towardsdatascience.com/using-python-to-push-your-pandas-dataframe-to-google-sheets-de69422508f
#https://stackoverflow.com/questions/45540827/appending-pandas-data-frame-to-google-spreadsheet

In [19]:
df_joint = (
   query_job.result().to_dataframe()
)


In [20]:
df_joint.head()

,value,transaction_hash,spent_transaction_hash,index,spent_output_index,block_timestamp,spent_block_timestamp
0,546,15293857e8ef9c383528bbd89cecad0ff9fc7e678de00b...,None,2,NaN,2019-07-30 08:35:34+00:00,NaT
1,37176,3d495d96794e03c3c46c9dade2210175c0835508fc94e7...,None,18,NaN,2020-09-05 12:24:48+00:00,NaT
2,0,457c80ccaed442db2313fb394d8c6684a991e77f924d78...,None,2,NaN,2019-05-01 08:36:26+00:00,NaT
3,2081670,6469c0c63c37e8635d77a6dd056c3ab9edf24afe5d80da...,6469c0c63c37e8635d77a6dd056c3ab9edf24afe5d80da...,0,0.0,2014-05-25 18:53:50+00:00,2014-05-26 12:51:47+00:00
4,2410000,ec971f4753523e0ee5d0c65090e2bc96c3c68b4ad96a97...,ec971f4753523e0ee5d0c65090e2bc96c3c68b4ad96a97...,0,0.0,2017-05-27 17:23:06+00:00,2017-05-27 17:38:21+00:00


In [23]:
!pip install gspread

In [24]:
!pip install df2gspread

     |████████████████████████████████| 61kB 3.1MB/s 
  Created wheel for df2gspread: filename=df2gspread-1.0.4-cp36-none-any.whl size=11955 sha256=c5d03fcb62b687d3bd3451e54610aff150165812f4c0b0298164a4b629aff762
  Stored in directory: /root/.cache/pip/wheels/19/d2/15/07b0a45a18edadbf0bb3a5655e997925e2b9cbf08b4a95342e
Successfully built df2gspread
ERROR: firebase-admin 4.4.0 has requirement google-api-python-client>=1.7.8, but you'll have google-api-python-client 1.6.7 which is incompatible.
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


In [41]:
import gspread as gs
import df2gspread as d2g
from oauth2client.client import GoogleCredentials
credentials = GoogleCredentials.get_application_default()
gc = gspread.authorize(credentials)

In [49]:
worksheet = gc.open('UTXO_Joint_Sample').sheet1

In [51]:
import gspread_dataframe as gd

In [53]:
gd.set_with_dataframe(worksheet, df_joint)

In [56]:
# TODO(developer): Set table_id to the ID of the destination table.
### Refers to 
#1. result large results:
#https://cloud.google.com/bigquery/docs/writing-results
#2. Export to dataframe
#https://cloud.google.com/bigquery/docs/exporting-data

table_id_joint_all = "crypto-291811.UTXO.joint_all"

job_config = bigquery.QueryJobConfig(destination=table_id_joint_all)

sql = """
  SELECT
    outputs.value,  
    outputs.block_timestamp,
    inputs.spent_block_timestamp,
   
  FROM `crypto-291811.UTXO.outputs` AS outputs
  LEFT JOIN `crypto-291811.UTXO.inputs` AS inputs
  ON outputs.transaction_hash=inputs.spent_transaction_hash  AND outputs.index = inputs.spent_output_index

"""

# Start the query, passing in the extra configuration.
query_job_all = client.query(sql, job_config=job_config)  # Make an API request.
query_job_all.result()  # Wait for the job to complete.


print("Query results loaded to the table {}".format(table_id_joint_all))

Query results loaded to the table crypto-291811.UTXO.joint_all


In [57]:
df_joint_all = (
   query_job_all.result().to_dataframe()
)

KeyboardInterrupt: ignored